This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_target_histogram(y_df, savepath, title='target histogram', label='target values'):
    """
    Method to plot the histogram of true y values

    Args:

        y_df: (pandas dataframe), dataframe of true y data values

        savepath: (str), path to save the plotted precision-recall curve

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.70
    fig, ax = make_fig_ax(aspect_ratio=0.5, x_align=x_align)

    #ax.set_title(title)

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=y_df)

    # do the actual plotting
    ax.hist(y_df, bins=num_bins, color='b', edgecolor='k')#, histtype='stepfilled')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))


    plot_stats(fig, dict(y_df.describe()), x_align=x_align, y_align=0.90, fontsize=14)
    # Save input data stats to csv
    savepath_parse = savepath.split('target_histogram.png')[0]
    y_df.describe().to_csv(savepath_parse+'/''input_data_statistics.csv')

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_df = pd.Series(pd.read_csv(StringIO('''
177.283020385
150.371025947
202.092457302
325.497887744
249.549443408
245.647988608
210.375206344
200.268881441
222.927320658
218.800570608
239.214774203
229.790128281
214.185752358
182.129032917
241.537702912
296.501210281
276.217368746
219.470530781
193.721693637
193.037892012
209.634412781
201.402112781
407.147186056
396.883335407
346.912901124
340.489474124
388.395179407
344.799275407
0.0
259.721126072
253.584854873
244.558933704
73.7475037184
79.884325527
64.766785692
188.139583805
121.202058532
146.594935546
143.14427
140.87646025
140.791488296
114.194239296
120.565464054
156.51523
110.462825
110.503751
99.671795625
120.15628375
94.573625692
129.676557394
106.566685144
96.279210144
95.946943894
95.343111144
241.991215895
214.252915662
185.182334608
186.835774858
202.290968412
200.047469412
181.659746314
176.193352581
145.936465692
304.216097984
71.5466155309
77.590214277
67.3902193795
178.590945618
111.989368657
113.270941671
149.58971125
121.012896
102.665207858
123.826342921
117.557011554
127.306710625
147.4918845
177.346326375
85.3211605625
130.710795625
99.387515567
97.753715269
101.679243394
88.069685269
89.1149867065
89.464121769
233.37177602
181.755921787
170.112525858
171.809960608
181.776187974
188.725323037
172.420203064
171.679717581
144.746720817
0.0
144.367702947
146.059828716
122.487016344
321.795002806
191.511615135
190.002071483
166.699388858
170.828354358
195.159839729
190.460889358
200.504538216
196.095876287
158.791748983
178.669885608
187.709914537
179.995444162
153.899276496
142.483392823
125.081234925
125.776102144
137.316409698
125.772960698
369.780134181
350.019596699
306.666897145
302.985862645
335.551879324
319.706164574
229.722452072
227.914068689
172.040538871
381.107806842
84.4124962184
54.9497902145
59.679202692
185.728826305
119.298373532
124.022946546
119.65758775
114.36792625
103.964993921
116.890894504
93.3858524289
137.488659125
209.8826285
127.740217
70.6664565
72.3673570833
77.0450205253
91.156936519
88.4608474773
80.9403347273
74.5855168107
83.809256644
228.914530895
187.427676662
165.793652358
156.941240858
174.631224037
187.51962462
168.464956314
158.367409518
121.204438234
291.955557984
131.350152405
70.2166732026
45.7531836013
37.9332462026
45.073198625
21.6105345
58.9939170208
109.47390069
92.4493164052
28.9476692026
0.0
63.0457358456
38.7490899228
23.5011419614
24.184234
24.8926086667
25.66391325
0.0
121.571007147
76.6050637187
183.350687466
97.8662705723
118.303763453
163.185749216
171.608093341
87.3132966495
120.214815608
390.4260105
95.1442246079
121.368421525
236.177315358
270.007912929
293.7439885
83.920796936
98.5115661808
108.66685467
70.9734215852
417.7519315
99.0662208891
129.128414087
254.265050483
291.384615991
66.0766101048
86.0496631495
97.0462161943
111.442293772
60.3566925184
73.0199955631
89.7212986079
100.971120483
106.61645203
636.339324716
156.229729822
236.664681953
408.488586216
515.805330466
209.8692985
71.9005615423
73.9283453934
122.389663096
132.814980798
68.6792013548
67.1398750184
272.8043925
78.6712295423
88.1149313934
157.715585096
178.235738798
86.4303692157
80.7377666536
84.202505616
92.5437335409
89.5029614657
227.491284799
113.135302229
136.316076767
175.379625843
201.81756512
122.31135567
274.349273966
103.801110549
134.180693408
198.061859124
237.521274841
278.307781849
102.036436282
128.364344873
182.740912055
238.334229236
8.216444
73.4978297298
71.4456317684
63.3627358456
163.297864937
32.0094037038
0.0
0.0
643.729766
127.681884176
179.258723131
347.3831
7.667046
69.4361417684
57.3220058456
92.4561079091
34.7342257044
29.459108777
0.0
90.406833621
75.8782445399
69.2869613885
41.601287871
156.627930263
63.6890316765
49.3432856618
9.7967661695
158.218632333
30.403271269
5.9009611652
0.678471
114.525327049
56.1529987529
36.5282198147
7.4876619381
56.6738366667
36.3631922476
12.9458568039
0.0
50.0993
54.2920320481
27.313536405
0.0
15.621525
0.0
0.0
0.0
141.664751466
73.8023139057
63.1512971348
64.5049606079
357.160728765
101.797990325
128.57645296
197.421378228
261.724803496
0.0
64.6627457298
53.5777137684
40.029931807
26.8896498456
0.0
0.0
19.4608832651
56.4799568664
74.586104069
29.163207289
78.0236106166
149.611167272
0.0
0.0
1.1676399614
53.828979
130.86141575
307.83053925
0.0
0.0
10.5881855
22.6412638903
66.1459738193
140.134457153
0.0
0.0
29.91808225
80.30888725
31.129048557
79.3342931526
177.633032344
177.441803716
96.8586983579
51.8644461789
70.3541023579
236.699858667
74.7720530417
88.1834983579
112.0035505
127.239966608
139.599683716
50.4901918579
0.0
212.302521091
127.825789025
149.85009451
198.106664217
206.158236245
288.486562824
119.77883703
151.797440519
218.946397498
253.560445966
289.712977799
118.302657605
145.197831669
202.689679797
251.771743647
377.502008598
119.774983252
150.472708246
219.402133514
294.89317499
59.6349893815
117.650529858
50.0254523342
75.1559557631
59.5521104052
70.8357719052
0.0
0.0
0.0
71.2267570302
33.9191772177
41.1709055302
43.192815874
59.0546828427
33.1384553579
45.2159618105
34.4706557296
60.1231125539
40.9146349171
39.3315709289
0.0
0.0
0.0
27.9926398279
29.9490807506
48.9115430065
47.0181371079
55.7153253105
25.8389489052
16.6684097187
62.1679659375
0.0
0.2039726667
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
4.514301375
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.6081073456
0.0
0.5934372329
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.9209429526
0.0
19.901478625
0.0
0.0
0.0
0.0
0.0
27.030439173
0.0
0.0
0.0
0.0
0.0
94.5062097102
169.769380462
73.9741468307
139.370003947
25.3695907381
32.0782685125
85.9609736131
89.2356505937
26.8282538631
33.56831065
12.6328149464
15.4429064062
17.3373103214
22.3715508958
75.3897667571
135.168012577
11.847970823
16.5750555745
26.3184750776
1.747132
0.0
0.0
0.0
0.0
0.0
34.9408400761
60.3605981789
108.224668483
33.436112274
7.7455884766
25.1748038261
40.6187326789
67.0851014829
6.3713860714
43.3637207337
32.789930692
38.146165692
53.842554067
60.521912442
0.0
0.0
56.751733567
65.031271442
40.254439817
40.835433942
43.488231192
44.850516692
32.1439513587
39.241275192
36.507512418
48.994329144
26.702212596
43.125215418
49.847735144
40.2646946055
43.995693519
0.0
0.0
0.0
31.343352695
19.2047596979
0.4420737039
39.226445692
162.308582145
210.808929074
178.001455145
232.209208574
157.374363216
181.910774716
0.0
161.615855778
189.644509841
131.304680028
133.559658341
140.198318684
150.508185653
155.792869145
194.268536574
108.979857614
86.4067635131
114.247314439
95.7636771618
115.200293626
102.514385537
0.0
0.0
0.0
100.457044713
69.2693877098
128.935951716
14.0944440539
42.2958931079
30.4689790539
87.3607131079
0.0
0.0
30.22875
29.653255
0.0
0.992490625
0.0
0.0
0.0
0.0
11.9314874914
49.3632299829
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
5.0562563395
34.3690176789
89.9135403579
9.5235200865
0.0
0.0
0.0
16.2043591789
44.4184733579
0.0
14.8100515625
13.804608375
8.970625875
0.0
52.6296356875
60.680276625
34.5198214375
34.149398125
55.3782776875
62.110810625
35.5034056875
40.797316625
42.324806125
50.8263675
7.3789669861
0.0
88.3802996944
141.470543625
23.94141725
74.7468692031
117.125778375
62.9442200833
79.2325764722
118.542972875
28.81237225
25.722252
24.13350925
22.18427375
65.3557068672
95.520522806
37.348833846
122.038739417
208.989074875
39.83120675
30.17024475
53.85203387
79.64651974
33.200068125
21.88033825
44.856381
45.433214
0.0
0.0
47.619279875
47.26926175
35.46209575
26.169518375
13.39023875
29.1224504136
23.6626028272
57.4995003125
61.213702625
31.378064
36.6852424375
48.571186875
49.15732575
35.123221625
35.15839525
28.2257425
0.0
0.0
0.0
86.889654024
36.179510048
73.395222096
73.0016409375
117.714483875
104.542615
36.257048
157.595599628
119.114167692
137.408712525
150.999876359
105.693165317
144.194571525
78.0094957753
93.9602340253
53.5428595253
107.975066775
124.630840359
86.5356568286
132.109024281
80.4591079773
112.573845031
65.294810373
100.859781275
77.9804427868
123.754016388
26.8325976181
85.1942082455
58.6508557954
75.8854938579
0.0
0.0
98.580755152
0.0
3.5098285625
85.2336429375
100.949224712
129.323671733
187.39104625
126.886312072
175.579596453
278.984665216
185.404210656
283.906453358
468.0346095
0.0
0.2022891806
138.892854722
22.478400277
59.183823902
151.553671152
21.172087375
52.4822445625
137.126058937
73.18129625
125.292207611
126.881898834
170.388617304
0.0
89.81673775
69.972385481
0.0
87.7843631349
93.6717096162
122.31135567
15.007828375
19.340972
0.0
0.0
12.44807725
0.0
0.0
8.691789
130.59739217
32.9412986079
47.0862504559
4.3671768848
93.4258071256
127.378189206
171.956702719
114.554644486
166.187428867
258.665309564
158.427043069
276.028720831
285.948164375
109.033410819
106.70233503
69.4735155302
257.075962699
284.898189574
298.368831324
263.83957477
276.340268528
232.454010858
83.917195748
44.5414483407
0.0
0.0
99.7625282026
48.464530625
185.524542312
52.0286768333
251.36028695
207.515713527
39.9617942
294.4157031
280.87646981
77.8684422026
47.369345875
165.778433187
0.0
139.763133125
197.246256937
42.1689745
211.4607162
202.664553441
31.5202492
281.102442575
286.014919019
126.7773445
202.283948375
175.569397812
230.921904608
140.538550375
229.064682937
134.5064424
321.98957475
139.345081333
367.6244316
299.692484775
199.684568608
128.448865625
211.476823813
62.4940495065
113.901486608
49.6917624592
73.1341625131
48.0411705302
50.8392286552
0.0
0.0
47.4586630927
57.3822137802
25.0244873427
25.3031122802
47.801375999
35.0768895927
38.1465154829
64.9749185605
22.3740347177
29.1802070302
13.1808016552
42.2355879796
36.1695635539
40.8070147026
0.0
0.0
0.0
26.2303387536
29.3008151019
32.9759565065
28.0575506079
95.3324888105
22.4196124052
33.6549837917
100.440194583
13.8924376789
51.4231023579
0.0
9.106651
0.0
0.0
0.0
10.51108275
0.0
1.40097275
0.0
0.0
12.1655324167
54.4675418333
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
21.517473096
0.0
0.0
18.4401941079
0.0
58.8853196728
78.1817223456
55.4728596164
72.0108022329
51.238815375
51.31971375
0.0
0.0
52.565170625
52.52667425
44.848887375
44.92360775
47.141973375
48.18377975
48.3755925539
58.5465181079
45.5261914257
47.9204658513
30.80780525
17.0719435
0.0
0.0
0.0
0.0
48.100167774
50.948668548
47.545535527
54.0734040539
69.8113911079
44.549417
155.465078116
234.8007094
132.773863413
204.099993472
75.4658120833
90.00434745
67.7439449583
78.3502295312
76.7577252083
91.3811395875
64.5192862917
74.5637353438
69.4770316667
82.9266298333
135.991885163
202.393591514
56.5407125895
60.6316355776
66.6463829526
44.2032278125
38.123014625
26.27533825
46.33040675
43.1541225
37.473804
82.6646597145
110.052006054
157.158326358
54.666157524
58.736874048
58.332307096
76.8321234645
97.0888905539
129.976759358
50.298441
51.3090163587
33.273366692
37.033060192
50.708099692
56.733026192
0.0
0.0
53.390279192
60.862885192
39.542985442
39.693297692
41.664526817
42.815880442
28.249643567
34.011113942
41.2956968437
45.2879704953
0.0
34.251060668
31.351448144
40.760422418
44.324421644
52.054920096
0.0
0.0
0.0
65.031540945
18.9559086979
39.207923192
162.89217552
214.364169074
169.71976527
224.402165324
138.758173341
150.562981466
0.0
0.0
141.471165903
154.697466591
122.980240153
124.729865091
128.386128809
134.671642403
152.42942927
195.442993324
102.151271114
81.5909270131
94.6538849388
83.0269046618
106.701246689
93.8711281618
0.0
0.0
0.0
91.3449974628
60.7251297098
0.0
120.596615216
65.9849631789
94.3019058579
81.0394981789
141.373725858
36.1816244687
43.1287284375
26.491269125
26.50926775
40.0673869375
49.581503375
24.413617625
25.08521475
26.749646375
28.58352225
63.3172566164
102.395742733
31.1645319882
36.3567934763
42.9040664526
17.6733958125
10.352771125
0.0
25.7130075625
27.595994625
28.72421875
51.7559553395
82.2953901789
140.515009858
33.6737583365
42.749246173
51.950721846
47.2433760895
71.6777316789
110.716942858
23.6410205
10.17451175
11.5904985
19.2994335
21.819592
66.761333125
95.226843375
42.3440435938
55.0100621875
71.1848256875
103.2663285
44.0103778438
54.8277306875
52.9102782812
70.1397815625
0.0
0.0
94.0584551875
137.881939187
14.001149625
57.6088494687
95.4966739375
49.4252902917
85.8908576875
105.442368438
19.808604625
17.7187702813
0.0
5.435506125
68.9092871406
80.6894183685
35.728066221
125.215369687
178.137720437
23.228189125
32.611025
18.9735033
57.144381475
8.6307090469
21.1232453125
0.0
6.8901275625
0.0
0.0
0.0
7.0559253125
3.6982160938
23.2118070625
0.0
9.3649023125
0.0
0.5317663897
11.2780820234
54.0607889219
69.00741775
0.0
0.0
0.0
0.0
5.2067181719
26.1974495
0.0
0.0
0.0
7.5656417818
35.6929084386
42.187522471
44.5467226484
73.7535701719
103.74396875
0.0
133.030324873
120.413788935
129.601532228
142.333953646
159.117704912
191.113049013
244.348237216
168.517826662
179.412889318
155.663046482
157.242981487
136.431443748
182.075420764
185.197855252
245.692438878
250.252038324
260.129737216
217.25282394
202.269808449
265.270592068
209.019844705
215.604849978
189.093084477
193.229456332
132.871922401
179.65413218
207.904806722
234.660650824
286.026589026
183.828681079
165.118221299
168.86730174
157.972802935
158.03796501
186.40027518
213.314659502
275.656178145
378.45337133
361.579777229
287.661839026
367.138367404
342.040019378
366.50308111
336.224696788
276.006928145
88.3272115282
90.3169483407
91.5340043601
106.514784005
99.6813432933
116.180976537
134.673116573
178.939784859
110.139317787
97.7903793826
114.272222612
81.5746890313
113.869161889
142.295817585
187.875800187
252.767162288
241.664953145
181.295284859
243.428371003
224.622870574
249.72472862
230.118335809
196.074158571
114.40771587
131.761462182
129.607779576
153.246503495
186.026530427
114.507151882
130.347187757
118.103266388
136.293767132
189.952208134
111.920470781
124.477908026
126.700822407
244.148911276
189.824629552
114.498161703
122.772579444
130.390352052
149.948332694
189.88100424
103.756754539
100.207954132
220.003380817
124.314123959
139.013580709
155.694636864
259.886083959
235.409224918
150.67246806
163.79917481
170.627480966
285.34442806
292.390663121
192.284656263
169.625113013
220.161511037
218.442616263
196.409133579
98.1125625013
100.864883177
144.708712621
152.280199029
191.836673799
123.246629227
122.677656829
141.535229704
144.0065577
133.95472581
140.405957382
190.075255435
87.0459946888
106.220358751
129.342617445
212.302508251
178.58616751
83.0462436023
101.768107977
122.431670219
205.092926144
210.33572768
103.165152778
121.428977278
156.136056722
231.481855293
231.128112002
118.248559781
135.390845031
177.752167907
251.196620227
319.116630645
154.419623787
169.573330537
292.536083787
351.817501878
236.33802852
250.57926852
267.340579878
367.183227895
357.145351324
237.062344466
262.896301216
273.613179324
362.461054466
294.310413121
177.346156263
191.512363013
222.246511037
226.507616263
323.82263694
206.364163582
221.055903582
237.99271494
340.306362957
309.856621449
189.648364591
204.046571341
223.816699449
320.529824591
340.437655068
213.49368171
227.87867171
255.478983068
349.453381085
318.399907705
201.32142318
232.699818764
233.810235705
337.214376097
199.00234553
187.894408571
101.887891775
59.8511380874
143.517205481
76.4960376187
205.564030427
111.026327787
129.319113662
156.185442293
76.9784094743
180.660958134
70.4378966858
70.822087223
130.558998313
82.3163755428
197.124129552
110.209087608
96.9915119328
149.647777957
210.107835323
195.67275424
91.2847571679
97.7091113525
212.756630817
83.5495498644
135.164756614
166.21281277
88.6267598644
247.684974918
151.356143966
161.293350716
208.659156871
252.161353966
296.095913121
177.317832168
184.516538918
247.005095074
280.854292168
206.610133579
96.9267962097
103.456135482
154.641811103
192.088746793
198.592673799
100.117426727
98.3944607723
141.921101323
193.969855727
88.4773613525
140.265181764
187.885254882
192.712755435
94.433508918
100.763646126
135.408932958
188.51316397
192.01041751
94.837602144
95.4547320607
131.977595034
183.449190081
213.33322768
102.79458377
108.425979583
159.401405204
197.309000251
234.968862002
116.278718092
121.127398975
185.807039526
216.454612644
313.722880645
154.707610893
160.233987758
240.576558169
259.086381287
354.737751878
253.019864472
259.216571222
329.042834288
353.349574472
364.707101324
238.522270371
245.488977121
315.514875145
338.955480371
330.939913121
179.999332168
186.717788918
250.995845074
144.889042168
332.21213694
252.001050484
258.640046797
317.205543003
350.052570547
317.356871449
230.917642395
238.98817827
298.525792574
328.54172252
349.605405068
247.474246653
252.611495932
323.63415062
346.902034252
346.087907705
218.137284733
223.307826541
305.414507809
318.43464993
164.002237758
120.403529576
141.453455481
71.0448978223
85.4936441348
89.577908623
142.826192293
78.9063338342
84.9348697092
104.459645435
124.842572407
144.154498313
68.4009027332
79.9859482049
88.5969514549
134.599602052
150.042027957
55.9498436559
65.6442278493
170.005981099
138.575931764
148.873886864
179.65931277
79.3248059118
89.5472626618
109.381015912
182.220980966
213.365656871
97.3721500131
118.314106763
123.035360013
223.624761037
244.800345074
136.443088216
139.928044966
147.525298216
162.784212621
151.898311103
60.2850907134
64.0241718081
55.5045541958
165.109729704
139.231351323
59.5241562349
64.5557198987
158.202912866
138.09122581
54.6384029447
54.158767202
77.4665625486
133.883367445
132.392432958
53.128244668
61.576363168
150.801863418
126.818170219
129.155595034
45.293214144
56.852612394
143.519281311
160.434806722
158.413405204
69.2085003268
72.8852659094
165.45646046
180.035167907
184.330789526
83.1507295945
86.8301581013
184.907725394
199.57809553
238.535058169
117.792276816
125.571143607
233.383665668
272.871579878
324.945084288
193.51782743
202.52553418
210.32978743
283.042429324
312.115625145
178.519868287
184.420825037
299.112578287
227.717761037
247.909595074
132.211588216
144.135044966
148.340548216
242.96546494
313.927543003
191.404536145
200.760492895
304.851246145
228.169699449
295.582042574
173.746535716
183.572992466
287.755495716
251.703483068
347.29590062
204.554893762
204.542100512
318.785353762
271.906597528
956.831955747
845.337300308
857.231665308
950.227905877
151.155130357
161.637649904
209.325438998
142.416121086
128.639131361
181.019901912
110.4724554
120.15829999
135.42648917
105.932372244
84.2083836779
98.8049065454
101.883875697
95.7786405842
107.655170358
67.7164974253
82.1190201348
123.367069795
81.0050963674
92.706468019
138.844995876
78.215252519
96.9622803223
141.354043251
57.4165433628
51.9883620098
46.578206626
77.0898973223
84.6282081436
104.682468001
110.065846334
143.722365881
190.046404974
109.913096508
131.79861623
194.691059745
105.83615073
115.142224674
151.08912256
94.5208382205
82.8280996542
92.7983292286
94.3913416736
95.4271065605
108.312182269
126.95531231
130.568581857
178.159873417
125.309562485
122.028582207
189.42037165
106.913116707
122.18519065
156.518088537
90.4495541968
83.2175656305
83.5342519118
102.21705765
96.4735725368
110.480194179
39.0465522292
68.9800767672
125.312625843
36.006686282
62.8478448729
117.389412055
18.9038755068
26.6189733223
46.8826689535
9.9473356615
12.2338936318
27.2145095723
17.4303139057
29.1372971348
35.1009606079
33.1481760595
51.8182329032
93.1208465907
37.0668895282
60.7829098407
113.321950466
44.3281811532
70.9524930907
124.273116966
24.231721997
32.6473247782
62.0242803407
49.6708469032
73.4595745907
112.909302858
43.4380611004
71.9683459539
127.66464503
41.8854296801
68.8773537441
125.171931241
30.5328625098
43.0159281418
72.1745594057
18.2785945327
22.0249128184
42.889278759
29.5478330923
33.183890824
56.6837961704
48.5930390252
75.8528445096
131.671414217
48.8429076049
75.9835816689
133.635429797
41.5835904347
58.5446973285
94.2635785923
27.2175724575
32.7734113742
58.3780479457
42.904352279
51.2762345131
77.5933817329
161.563183779
173.730028128
205.075216824
162.590223425
176.433607419
234.937875407
149.540965092
187.848590753
147.336842074
135.840083404
121.553577378
102.112315324
140.218828436
124.01781744
95.2060454486
173.385595035
174.909068638
211.335265844
178.369049186
184.21597694
197.176332449
188.271436029
198.013500628
210.616629824
160.456726873
146.552082315
113.527293199
184.779292217
185.414463003
167.324804574
170.292754279
182.101848628
211.267959969
172.754543925
187.207177919
236.861913524
163.409285592
163.793911253
165.127412574
146.552153904
133.312147878
111.384135824
154.672898936
142.93338794
115.407865949
181.387574779
192.128669128
218.074203115
185.543114425
200.391998419
220.40870164
179.710356092
184.787731753
189.998651657
159.787224404
147.483468378
122.370956324
171.603969436
163.98220844
137.406686449
54.9152362749
33.2804978342
0.0
30.7711248333
29.8870605278
77.3860276912
124.54211081
91.4620807359
108.537383781
123.93596706
159.403545859
83.5686241725
90.6089706538
100.815663616
271.319339431
114.754340135
137.479652578
174.060277466
106.745758739
113.060643677
105.722179614
109.488955989
113.886203609
102.775420338
96.0360383765
86.699751058
258.828050931
110.285835802
135.954460139
183.271284153
122.316042763
117.247059064
117.249168138
126.798490013
137.791901025
97.9185800222
104.37297146
104.244207537
296.630230431
113.092107877
137.583440952
184.901448591
188.514712621
184.59131561
185.661693755
183.151200043
164.660390859
185.05127539
171.490363314
156.027039162
397.239965431
208.035658149
237.583378832
291.099073324
112.287386278
111.066721692
117.140514423
115.594392567
117.143168942
113.501498734
110.864717892
109.297299507
98.7844184122
298.245707431
135.955482743
162.41908077
216.122194574
138.582903341
145.310468668
140.364781485
141.96565963
144.111165918
136.808520882
137.369484955
135.369316569
121.692419186
335.321293431
163.497124024
191.31459627
244.717542324
89.81673775
91.2669747157
87.6081324032
88.6858955907
93.7481719657
90.748756192
82.8791005565
79.8648318973
69.7752945789
263.543371431
107.783426118
134.35498302
182.613119716
29.7477073211
42.1335069877
66.9631063211
37.1426186397
56.511329625
66.9909409167
33.3198729476
44.1815882408
183.475518827
28.6171985101
36.3667393658
290.221571077
34.8221973851
46.9172371158
197.610066577
21.5562064476
27.1330052408
71.0907635772
118.9471195
283.093127667
22.3792303226
25.2930529908
87.1299483272
15.1299348382
4.4579620221
20.0619416351
19.1667256158
95.3050435772
31.3127508851
37.3225941158
38.6328698346
59.7413320147
95.049006375
72.9422615625
109.582963917
113.170480976
39.7988018658
59.1429460772
95.6649845
112.949106484
165.733078594
198.665046858
50.4471564427
71.4796191771
72.67526425
49.7936119168
78.4670661792
134.244898562
40.6890700892
61.6998797448
98.8464407187
28.4417281351
35.2832986158
35.0459395772
105.859697484
143.075474781
140.737464875
52.8734187279
93.8114346912
140.488835358
63.5864629799
99.8669764304
107.860026374
111.369551266
192.009283406
341.670362
10.4314375689
0.0
0.0
109.972952484
143.739983969
161.966268358
87.0884028594
101.020673844
94.3179777045
132.019981484
182.417159594
179.369513621
129.167628484
178.944559094
182.449930039
38.749668158
38.1360533299
35.0703869833
95.5308332344
123.689710284
110.110223375
84.5812328594
97.2556960313
90.435045
0.0
0.0
0.0
62.0947510267
98.2564415885
155.909864281
49.5323893594
66.2624423437
82.91347525
6.9223966347
0.0
0.0
0.0
0.0
0.0
64.1409236857
93.8423028603
112.1749515
71.888596023
124.759380625
198.105435858
24.1527620726
21.5058664908
4.7555753272
17.8464076544
113.483054382
136.252989858
62.3247673342
93.7906657631
49.2192869886
56.3388158052
0.0
0.0
56.3192893526
59.2192098
54.1514922177
45.4043655302
53.285630874
49.4793928427
72.6725203579
100.47292181
73.2347672296
62.4481655539
34.2214622026
160.334632542
121.974896179
28.3696734526
0.0
0.0
0.0
74.3588962714
39.7951736375
0.0
51.9157130065
61.5578071079
48.7758689052
101.755733
160.8491605
43.4643305539
63.9648556079
21.353923875
28.48129225
0.0
0.0
21.28138975
28.314724
26.35395975
33.885364
9.3192484688
2.5504414375
61.119531625
93.06800775
21.5324071757
28.6637588513
12.4011708125
11.017786125
7.99376675
0.0
0.0
0.0
23.5812529198
32.3684503397
28.407001277
37.3769470539
14.2008055
84.7289651789
120.026452358
45.6348093039
74.8181406079
27.129298625
34.10036925
0.0
0.0
27.119653875
34.01532975
31.763370625
38.65176325
29.915706625
37.78468525
49.2018258039
70.7659236079
27.7992143632
29.9034507263
17.2121126875
14.178747375
0.0
0.0
0.0
26.7038101073
33.7558922147
37.4680233395
58.1498186789
20.138728
184.387404762
267.854425525
139.06215854
207.75718308
75.6574945
81.726855
72.650127375
75.14562075
76.5032664308
82.99657195
76.8437187083
79.0798034167
77.2377140833
82.9015441667
152.154945491
221.768006983
68.9710696132
69.8177552263
67.4148764526
62.3324679375
55.140801875
40.70021975
60.54108325
52.2740325
36.096681
91.6707429645
116.752351929
166.797819858
70.4541653573
70.5986967147
65.5202594293
82.2454137145
97.2311934289
121.693502858
69.473884
91.1059876087
38.1925113587
39.1108878587
45.499570942
52.1415070253
0.0
0.0
46.950750442
55.0886160253
43.821956692
45.0377785253
42.385498067
44.4093612753
48.240040192
52.5504455253
38.1742435291
33.9806021996
40.8206874388
43.945490019
46.0103451793
38.3815527513
39.127470644
40.2393064293
0.0
0.0
0.0
56.665649793
103.731414727
38.8036348587
152.57088652
190.870878449
123.849520145
166.347668574
86.115789799
84.2613186157
0.0
0.0
87.2670421631
86.5192126105
82.6202450282
73.3023683407
82.5826336845
75.6793956532
123.233184145
157.611746574
70.9430276144
53.3006835131
17.7019953105
67.6609259387
71.7619801618
37.6093386079
110.664773689
105.624925662
36.2234796079
0.0
0.0
0.0
90.1656399358
31.5229081559
0.0
87.8388717157
94.6025620331
126.616163691
84.8759223664
142.323884358
34.8683784583
41.137907275
28.0724115312
27.2516126875
37.4352793437
46.3580983125
31.5042600312
30.7458096875
30.5960387812
31.2618671875
80.1722752414
120.865840108
37.4221989413
45.9371875076
57.8409146401
29.3690972656
29.2307341562
24.5125079375
27.1567125781
25.9562147812
20.7374691875
59.0626222926
94.0057842102
160.395108045
36.6110446855
42.6966289959
46.7667976168
54.0260430426
80.1561257102
120.061041045
28.416710375
66.501248625
70.04323975
17.0003038125
0.0
45.407611625
51.90442725
38.778547375
35.37179875
53.686253625
52.42546125
45.062381625
46.69821725
45.3272820625
50.027518125
33.46241925
22.8817925
89.71351025
125.245006875
7.0009685
78.06700275
92.808241625
54.3341091667
77.68295
99.023686125
13.4709235
27.90730325
19.9708105
5.974325
65.7797960302
78.303986056
25.115635096
121.777150396
173.113038125
21.912008
38.754046
33.004800495
71.75494749
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.5885125938
8.6688716875
289.029515
0.0
0.0
0.0
0.0
28.8957797708
0.0
0.0
0.0
0.0
36.5545469302
33.5261903605
19.410199346
0.0
47.2079029375
23.809066
0.0
''')).iloc[:,0])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/learning_curves/learning/target_histogram.png'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_target_histogram(y_df, savepath, label)
display(Image(filename='target_histogram.png'))
